In [1]:
import pandas as pd
import datetime

In [2]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

In [3]:
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"]) #formato fecha
df_vp["mes"]  = df_vp["fecha"].dt.month #agregar mes
df_vp["ano"]  = df_vp["fecha"].dt.year #agregar año
df_vp["dia"]  = df_vp["fecha"].dt.day #agregar año


pd.set_option("display.max_rows", None)   # muestra todas las filas
pd.set_option("display.max_colwidth", None)  # muestra texto completo en celdas

df_perdidas = df_vp[df_vp["cantidad"] > df_vp["cantidad_existencia"]] # ventas peridas realmente
df_perdidas["vp"] = df_vp["cantidad"] - df_vp["cantidad_existencia"]


# REGLAS FLTRADO DE VENTAS PERDIDAS
vp_filtradas = df_perdidas[(df_perdidas["cantidad"]>=1)  & (df_perdidas["cantidad"]<=99)].copy()
vp_filtradas.sort_values(by=["vp"], ascending=False).head(5)

vp_filtradas = vp_filtradas[~vp_filtradas["cliente_name"].str.contains("PROMOCION", case=False, na=False)].copy() #Solicitar nueva regla retirar ventas perdidas para cliente promocion esta duplicandolas

vp_def = vp_filtradas[["store_name","product_id","origen","cliente_name","fecha","vp","mes","dia","ano"]]

/tmp/ipykernel_64882/1173187793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perdidas["vp"] = df_vp["cantidad"] - df_vp["cantidad_existencia"]


In [4]:
import ast

def get_second(val):
    try:
        # convierte el string en lista real de Python
        lista = ast.literal_eval(str(val))
        if isinstance(lista, list) and len(lista) > 1:
            return lista[1]  # devuelve el segundo elemento
    except Exception:
        return None
    return None

vp_def["product_id"] = vp_def["product_id"].apply(get_second)

import re

def split_cliente(val):
    if val is None:
        return None, None
    text = str(val)
    match = re.match(r"\[(.*?)\]\s*(.*)", text)
    if match:
        return match.group(1), match.group(2)  # (ID, Nombre)
    return None, text

# Crear dos columnas nuevas
vp_def = vp_def.copy()

vp_def.loc[:, "cliente_id"], vp_def.loc[:, "cliente_name"] = zip(*vp_def["cliente_name"].apply(split_cliente))
vp_def = vp_def.drop(columns=["cliente_id"])

/tmp/ipykernel_64882/3119152225.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vp_def["product_id"] = vp_def["product_id"].apply(get_second)


In [5]:
import re

def split_producto(val):
    if val is None:
        return None, None, None
    text = str(val).strip()
    
    # buscar con regex: [producto] descripcion (codigo)
    match = re.match(r"\[(.*?)\]\s*(.*)\((.*?)\)\s*$", text)
    if match:
        producto = match.group(1).strip()
        descripcion = match.group(2).strip().rstrip(",")   # quitamos coma final si la hay
        codigo_interno = match.group(3).strip()
        return producto, descripcion, codigo_interno
    
    return None, text, None  # fallback: si no cumple el patrón

# aplicar y crear nuevas columnas
vp_def.loc[:, ["producto", "descripcion", "codigo_interno"]] = list(vp_def["product_id"].apply(split_producto))
vp_def = vp_def.drop(columns=["product_id"], errors="ignore")


In [6]:
vp_def.head()

,store_name,origen,cliente_name,fecha,vp,mes,dia,ano,producto,descripcion,codigo_interno
33,SUCURSAL BARRANQUILLA,cotizacion,CONSTRUCCION Y CONSULTORIAS DE OBRAS DE INGENERIA URBANISMO Y ARQUITECTURA CONTINUAR S.A.S,2024-12-01,1,12,1,2024,DAB14570025,"DA4570A FILTRO AIRE 2_ DONSSON - PERKINS, CATERPILLAR",025 DA4570A
36,SUCURSAL VALLADOLID,pos,JEIMMY\t CAROLINA GONZALEZ SALINAS,2024-12-02,9,12,2,2024,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,189 GS740
37,SUCURSAL NORTE,cotizacion,CONSUMIDOR FINAL NORTE,2024-12-02,1,12,2,2024,BAE08201125,"DA8201 FILTRO AIRE HITACHI, HYUNDAI, JCB",125 PA31002
41,SUCURSAL NORTE,cotizacion,CONSUMIDOR FINAL NORTE,2024-12-02,1,12,2,2024,DAE02985025,DA2985 FILTRO AIRE HYUNDAI HD270,025 DA2985
46,SUCURSAL NORTE,cotizacion,CONSUMIDOR FINAL NORTE,2024-12-02,1,12,2,2024,DCE10580139,"G580 FILTRO COMBUSTIBLE LV150, ISUZU,HINO",139 G580


In [26]:
import pandas as pd

# Asegurar tipo fecha
#vp_def["fecha"] = pd.to_datetime(vp_def[["ano","mes","dia"]])

# Crear columna de Mes (primer día del mes)
vp_def["Mes"] = vp_def["fecha"].dt.to_period("M").dt.to_timestamp()

# Agrupar por sucursal + producto + mes
vp_mensual = (
    vp_def.groupby(["store_name","producto","Mes"], as_index=False)
          .agg({"vp": "sum"})
          .rename(columns={"store_name": "Sucursal",
                           "producto": "Producto",
                           "vp": "Ventas_perdidas"})
)

prueba = vp_mensual[(vp_mensual["Sucursal"]=="SUCURSAL BARRANQUILLA") & (vp_mensual["Producto"].str.contains("DAB025"))]
prueba.sample()

,Sucursal,Producto,Mes,Ventas_perdidas
7832,SUCURSAL BARRANQUILLA,DAB02567025,2025-07-01,6
